# Advanced RAG on Hugging Face documentation using LangChain

## Import

In [22]:
# Use faiss-gpu-cu12 or faiss-gpu-cu11 accroding to your CUDA version instead of faiss-gpu
!pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers openpyxl pacmap datasets langchain-community ragatouille matplotlib faiss-gpu-cu12

In [1]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

# This will be helpful when visualizing retriever outputs
pd.set_option("display.max_colwidth", None)

### Load your knowledge base

In [2]:
import datasets

ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

In [3]:
# In my case, I needed to update jupyter : !pip install -U jupyter 

In [4]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

  0%|          | 0/2647 [00:00<?, ?it/s]

## 1. Retriever - embeddings

### 1.1 Split the documents into chunks

In [ ]:
from sentence_transformers import SentenceTransformer

print(f"Model's maximum sequence length: {SentenceTransformer('thenlper/gte-small').max_seq_length}")

